## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-25-23-50-54 +0000,nyt,Prosecutors Fail 3 Times to Charge Woman With ...,https://www.nytimes.com/2025/08/25/us/politics...
1,2025-08-25-23-50-45 +0000,nyt,SpaceX Starship 10th Test Launch: How and When...,https://www.nytimes.com/2025/08/25/science/spa...
2,2025-08-25-23-47-14 +0000,nyt,A judge threatens to hold a Trump official in ...,https://www.nytimes.com/live/2025/08/25/us/tru...
3,2025-08-25-23-47-07 +0000,wapo,Chicagoans say Trump’s plan to send troops to ...,https://www.washingtonpost.com/nation/2025/08/...
4,2025-08-25-23-42-39 +0000,nypost,Hero American tourist and model reveals extent...,https://nypost.com/2025/08/25/world-news/hero-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2309/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
18,trump,48
51,new,12
285,will,10
241,he,9
90,deal,9


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
137,2025-08-25-17-11-00 +0000,wsj,President Trump intensified his tough-on-crime...,https://www.wsj.com/politics/policy/trump-exec...,112
75,2025-08-25-20-38-55 +0000,nypost,Illinois Gov. JB Pritzker calls Trump ‘wanna b...,https://nypost.com/2025/08/25/us-news/illinois...,98
30,2025-08-25-22-24-17 +0000,nypost,"NYC assault victim, 94, chimes in on Trump ord...",https://nypost.com/2025/08/25/us-news/nyc-assa...,97
107,2025-08-25-19-21-00 +0000,nypost,"Ivanka Trump’s father-in-law, US Ambassador to...",https://nypost.com/2025/08/25/world-news/charl...,92
202,2025-08-25-11-00-00 +0000,nypost,Trump to sign executive order ending cashless ...,https://nypost.com/2025/08/25/us-news/presiden...,90


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
137,112,2025-08-25-17-11-00 +0000,wsj,President Trump intensified his tough-on-crime...,https://www.wsj.com/politics/policy/trump-exec...
161,67,2025-08-25-15-40-13 +0000,nypost,Keurig Dr Pepper to buy JDE Peet’s in $18B dea...,https://nypost.com/2025/08/25/business/keurig-...
112,49,2025-08-25-19-11-00 +0000,wsj,Kilmar Abrego Garcia was arrested by immigrati...,https://www.wsj.com/us-news/law/kilmar-abrego-...
107,44,2025-08-25-19-21-00 +0000,nypost,"Ivanka Trump’s father-in-law, US Ambassador to...",https://nypost.com/2025/08/25/world-news/charl...
159,36,2025-08-25-15-46-00 +0000,wsj,Tech executives including VC firm Andreessen H...,https://www.wsj.com/politics/silicon-valley-la...
20,35,2025-08-25-23-00-43 +0000,nypost,Waymo’s robotaxis hit NYC in test run — but lo...,https://nypost.com/2025/08/25/us-news/waymos-r...
75,35,2025-08-25-20-38-55 +0000,nypost,Illinois Gov. JB Pritzker calls Trump ‘wanna b...,https://nypost.com/2025/08/25/us-news/illinois...
102,34,2025-08-25-19-40-01 +0000,nypost,Trump claims ‘good conversation’ with Putin si...,https://nypost.com/2025/08/25/us-news/trump-cl...
245,33,2025-08-25-01-56-00 +0000,wsj,Here’s what you need to know about the unusual...,https://www.wsj.com/politics/policy/texas-cali...
116,31,2025-08-25-18-31-54 +0000,bbc,Five journalists among 20 killed in Israeli do...,https://www.bbc.com/news/articles/cp89rp48246o...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
